# Use pulp to solve a wedding seating problem

In [1]:
import pulp

In [2]:
max_tables = 5
max_table_size = 4
guests = 'A B C D E F G I J K L M N O P Q R'.split()

**happiness function**
The happiness of the table is defined as the maximum distance between (sorted?) letters. 

In [3]:
def happiness(table):
    return abs(ord(table[0]) - ord(table[-1]))

**generate possible seating arrangements**

In [4]:
combos = pulp.allcombinations(guests, max_table_size)
possible_tables = [tuple(c) for c in combos]
print(possible_tables[245])

('A', 'K', 'L')


**create a binary variable dictionary to state that a table setting is used**

[LpVariable.dicts](https://coin-or.github.io/pulp/technical/pulp.html?highlight=dicts#pulp.LpVariable.dicts), a class method, creates a dictionary of LpVariable with the specified associated parameters

Each possible table setting will eventually have a not-used/used value of 0 or 1.  This method simply creates a dict with:
* keys = the values of the possible table seatings, 
* values = LpVariables with a variable name of table_&lt;key&gt;, and which can be either 0 or 1.


In [5]:
x = pulp.LpVariable.dicts('table', possible_tables, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

nthKey=list(x)[245]
print(nthKey)
print(x[nthKey])

('A', 'K', 'L')
table_('A',_'K',_'L')


In [6]:
seating_model = pulp.LpProblem("WeddingSeatingModel", pulp.LpMinimize)

**Set the objective**

Augmenting the model with a value makes this the objective.
Only one objective may be added to a model.
Since we said this was an LpMinimize type, the objective is to minimize this value. (?)

`pulp.lpSum(vector)` Calculates the sum of a list of linear expressions.

Parameters:  `vector` – A list of linear expressions

In [7]:
# print([happiness(table) * x[table] for table in possible_tables])
seating_model += pulp.lpSum([happiness(table) * x[table] for table in possible_tables])

Augmenting the model with an expression adds a constraint. The solution may not violate constraints.
We specify the sensible constraint that the solution has to be <= some limit.

In [8]:
seating_model += pulp.lpSum([x[table] for table in possible_tables]) <= max_tables,"Maximum_number_of_tables"

Now we need to ensure that guests are not duplicated. They can sit at one table only. Add a constraint for each guest.

In [9]:
for guest in guests:
    seating_model += pulp.lpSum([x[table] for table in possible_tables
                                if guest in table]) == 1, "Must_seat_%s"%guest

Tell the model to solve

In [10]:
seating_model.solve()

1

In [11]:
print("The chosen tables are out of a total of %s:"%len(possible_tables))
for table in possible_tables:
    if x[table].value() > 0:
        print(table)

The chosen tables are out of a total of 3213:
('M', 'N')
('E', 'F', 'G')
('A', 'B', 'C', 'D')
('I', 'J', 'K', 'L')
('O', 'P', 'Q', 'R')
